In [1]:
from scipy.io import loadmat
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymatreader import read_mat
import tensorflow as tf
from tensorflow import keras
from keras import layers
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import eeg_funcs

In [ ]:
filepath = glob('data/train/*.mat')
d_array, d_labels = eeg_funcs.get_comp2_array(filepath, True, True)
print(d_array.shape)

In [ ]:
d_array = d_array*1e6
print(np.min(d_array), np.max(d_array))

In [4]:
def normalize_data(data):
    data = (data - np.min(data)) / (np.max(data) - np.min(data))
    result = 2 * data - 1
    return result

In [ ]:
print(d_array[2875,:,5])
d_array = normalize_data(d_array)
print(d_array[2875,:,5])

In [ ]:
print(d_array.shape, d_labels.shape)
print(d_labels[2875])
plt.plot(d_array[2875,:,5])

In [7]:
#t_array = np.expand_dims(d_array, axis=3)
#data_array = t_array
#data_array = tf.convert_to_tensor(data_array, dtype=tf.float32)
data_array = tf.convert_to_tensor(d_array, dtype=tf.float32)

In [8]:
BUFFER_SIZE = 30000
BATCH_SIZE = 64
train_dataset = tf.data.Dataset.from_tensor_slices(data_array).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
discriminator = keras.Sequential(
    [
        keras.Input(shape=data_array.shape[1:]),
        layers.Conv1D(64, kernel_size=2, strides=2, padding="same"),
        #layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),
        #layers.Dropout(0.3),
        layers.Conv1D(64, kernel_size=2, strides=2, padding="same"),
        #layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),
        #layers.Dropout(0.3),
        layers.Conv1D(64, kernel_size=2, strides=2, padding="same"),
        #layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),
        #layers.Dropout(0.3),
        layers.Conv1D(64, kernel_size=2, strides=2, padding="same"),
        #layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),
        #layers.Dropout(0.3),
        layers.Conv1D(64, kernel_size=2, strides=2, padding="same"),
        #layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),
        #layers.Dropout(0.3),
        layers.Flatten(),
        layers.Dense(1, activation="sigmoid"),
    ],
    name="discriminator",
)
discriminator.summary()

In [ ]:
latent_dim = 256

generator = keras.Sequential(
    [
        keras.Input(shape=(latent_dim,)),
        layers.Dense(5*256),
        layers.LeakyReLU(alpha=0.2),
        layers.Reshape((5, 256)),
        layers.Conv1DTranspose(64, 2, strides=2, padding='same', use_bias=False),
        #layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv1DTranspose(64, 2, strides=2, padding='same', use_bias=False),
        #layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv1DTranspose(64, 2, strides=2, padding='same', use_bias=False),
        #layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv1DTranspose(64, 2, strides=2, padding='same', use_bias=False),
        #layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv1DTranspose(8, 2, strides=2, padding='same', use_bias=False, activation='tanh'),
    ],
    name="generator",
)
generator.summary()

In [19]:
class GAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super(GAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(GAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn
        self.d_loss_metric = keras.metrics.Mean(name="d_loss")
        self.g_loss_metric = keras.metrics.Mean(name="g_loss")

    @property
    def metrics(self):
        return [self.d_loss_metric, self.g_loss_metric]

    def train_step(self, real_images):
        # Sample random points in the latent space
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Decode them to fake images
        generated_images = self.generator(random_latent_vectors)

        # Combine them with real images
        combined_images = tf.concat([generated_images, real_images], axis=0)

        # Assemble labels discriminating real from fake images
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )
        # Add random noise to the labels - important trick!
        labels += 0.05 * tf.random.uniform(tf.shape(labels))

        # Train the discriminator
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(zip(grads, self.discriminator.trainable_weights))

        # Sample random points in the latent space
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Assemble labels that say "all real images"
        misleading_labels = tf.zeros((batch_size, 1))

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(random_latent_vectors))
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Update metrics
        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)
        return {
            "d_loss": self.d_loss_metric.result(),
            "g_loss": self.g_loss_metric.result(),
        }

In [23]:
class GANMonitor(keras.callbacks.Callback):
    def __init__(self, num_img=1, latent_dim=100):
        self.num_img = num_img
        self.latent_dim = latent_dim

    def on_epoch_end(self, epoch, logs=None):
        random_latent_vectors = tf.random.normal(shape=(self.num_img, self.latent_dim))
        generated_images = self.model.generator(random_latent_vectors)
        generated_images.numpy()
        for i in range(self.num_img):
            img = generated_images[i]
            img2 = img[:,5]
            plt.plot(img2)
            plt.savefig("Conv9testimages/single/generated_img_%03d_%d.png" % (epoch, i))
            plt.clf()
            plt.plot(img)
            plt.savefig("Conv9testimages/full/generated_img_%03d_%d.png" % (epoch, i))
            plt.clf()
            ##img = keras.preprocessing.image.array_to_img(generated_images[i])
            ##img.save("generated_img_%03d_%d.png" % (epoch, i))

In [21]:
checkpoint_filepath = 'checkpointConv9/checkpoint.{epoch:02d}'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='g_loss',
    mode='min',
    save_best_only=True)

In [ ]:
epochs = 3500  # In practice, use ~100 epochs

gan = GAN(discriminator=discriminator, generator=generator, latent_dim=latent_dim)
gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.001, beta_1=0, beta_2=0.99),
    g_optimizer=keras.optimizers.Adam(learning_rate=0.001, beta_1=0, beta_2=0.99),
    loss_fn=keras.losses.BinaryCrossentropy(),
)

gan.fit(
    train_dataset, epochs=epochs, callbacks=[GANMonitor(num_img=1, latent_dim=latent_dim), model_checkpoint_callback]
)